In [1]:
import pandas as pd
import numpy as np
import re
import copy

In [2]:
#open the file
cable_file = 'Diplomacy_W129.xlsx'
df = pd.read_excel(cable_file).T

In [30]:
class Cleaner:
    def __init__(self,cable_dataframe):
        self.df = cable_dataframe
        self.separate_rows()
        self.clean_all()
        self.cleaned = self.recompile()
        
    def separate_rows(self):
        self.turns = self.df.iloc[0]
        self.authors = self.df.iloc[1]
        self.subjects = self.df.iloc[2]
        self.cables = self.df.iloc[3]
    
    def clean_all(self):
        self.clean_turn()
        self.clean_author()
        self.clean_subject()
        self.clean_cables()

    def clean_turn(self):
        pass
    def clean_author(self):
        for i in range(len(self.authors)):
            self.authors[i] = self.authors[i].replace('the','The')
    def clean_subject(self):
        for i in range(len(self.subjects)):
            self.subjects[i] = self.subjects[i].replace('the','The')
    def clean_cables(self):
        bad_char = [',','!','.','"','\\',"'s'","'"]
        break_char = ['\n','\r']
        for i in range(len(self.cables)):
            self.cables[i] = self.cables[i].replace('. ','\r')
            for j in bad_char:
                self.cables[i] = self.cables[i].replace(j,'')
            for j in break_char:
                self.cables[i] = self.cables[i].replace(j,'. ')
            self.cables[i] = self.cables[i].split('. ')

    def recompile(self):
        final = pd.DataFrame(self.turns).T
        final = final.append(self.authors)
        final = final.append(self.subjects)
        final = final.append(self.cables)
        return(final)

cdf = Cleaner(copy.copy(df))
cdf.cleaned

,0,1,2,3,4,5,6,7,8,9,...,3708,3709,3710,3711,3712,3713,3714,3715,3716,3717
turn,2,2,3,3,4,5,6,6,6,7,...,499,499,499,499,499,499,499,499,500,500
author civ,Germany,Songhai,Songhai,Korea,Morocco,Morocco,The Incas,Germany,Korea,Songhai,...,Morocco,Germany,Korea,The Incas,Morocco,Morocco,Germany,Germany,Songhai,Songhai
subject civ,Songhai,Ife,Germany,Mombasa,Kathmandu,The Incas,Morocco,Ife,Bogota,The Iroquois,...,Germany,The Incas,The Incas,Singapore,Kathmandu,Ife,Mombasa,Ife,The Incas,Panama City
cable,"[To Foreign Secretary, , Hopefully this finds ...","[Dear Foreign Secretarys Office, , It is alway...","[To Foreign Office, , I hope this letter finds...","[To Honorable Foreign Secretary, , I hope all ...","[Dearest Foreign Office, , I hope all is going...","[Honorable Foreign Secretary, , Your humble re...","[Dear faithful foreign correspondent, , We her...","[To Foreign Correspondents Office, , We hope t...","[Dearest ambassador to Bogota, , We here at ho...","[Greetings Chief of Mission, , Best wishes fro...",...,"[Greetings charge daffaires in Germany, , We a...","[Dearest faithful foreign correspondent, , I h...","[Greetings Korean embassy in The Incan Nation,...","[Foreign Secretarys Office, , Hopefully this f...","[Greetings Ambassador, , I hope all is going w...","[Greetings foreign correspondent, , We hope th...","[Dear Mombasa Desk, , We trust that things are...","[Dear Your Excellency the Ambassador, , All of...","[To Your Honor, , I hope this finds you in hig...","[Greetings charge daffaires in Panama City, , ..."


In [31]:
class Dictionaries:
    def __init__(self,clean_object):
        self.cdf = clean_object
        self.define_attributes()
        self.get_placenames()
        
        self.generalize_cables()
        
    def define_attributes(self):
        self.data = self.cdf.cleaned
        self.turns = self.cdf.turns
        self.authors = self.cdf.authors
        self.subjects = self.cdf.subjects
        self.cables = self.cdf.cables
        
    def get_placenames(self):
        self.countries = self.get_countries()
        print(self.countries)
        self.city_states = self.get_cities()
        print(self.city_states)
        self.ethnicities = ['the Germans','The Germans','German ','the Koreans','The Koreans','Korean',
                       'the Moroccans','The Moroccans','Moroccan','the Incas','The Incas','Incan',
                       'the Iroquois','The Iroquois','Iroquois','the Songhais','The Songhais','Songhain']

    def get_countries(self):
        countries = np.array(self.authors)
        unique_countries = np.unique(countries)
        #unique_countries.append('the Incas','the Iroquois')
        return(unique_countries)

    def get_cities(self):
        placenames = np.array(self.subjects)
        unique_placenames = np.unique(placenames)
        unique_citystates = np.setdiff1d(unique_placenames,self.countries)
        return(unique_citystates)
    
    def generalize_cables(self):      
        gen_cab = []
        for i in range(len(self.cables)):
            for j in range(len(self.cables[i])):
                phrase = self.cables[i][j]
                for k in self.ethnicities:
                    phrase = phrase.replace(k,'COUNTRY')
                for k in self.countries:
                    phrase = phrase.replace(k,'COUNTRY')              
                for k in self.city_states:
                    phrase = phrase.replace(k,'CITY STATE')
                gen_cab.append(phrase)
        gcabs = np.array(gen_cab)
        self.cable_types = list(np.unique(gcabs))

        

        
dic = Dictionaries(copy.copy(cdf))
for i in dic.cable_types:
    print(i)

['Germany' 'Korea' 'Morocco' 'Songhai' 'The Incas' 'The Iroquois']
['Bogota' 'Budapest' 'Florence' 'Ife' 'Kathmandu' 'Kyzyl' 'Mombasa'
 'Panama City' 'Singapore' 'Yerevan']

 A competitor is how COUNTRY diplomatic cables characterized their dealings with COUNTRY
 A competitor is how our spies characterized their association with COUNTRY
 A friendly attitude with CITY STATE is what we assert
 A neutral civilization is how our spies defined their dealings with COUNTRY
 A neutral disposition with CITY STATE is what we avow
 A neutral manner with CITY STATE is what we assert
 A neutral party is how our agents defined their relations with COUNTRY
 A neutral posture with COUNTRY is what we state
 A neutral stance with CITY STATE is what we assert
 A protective position with CITY STATE is what we state
 According to COUNTRY sources the COUNTRY are largely about unknown preeminence
 According to our agents COUNTRY are focused on cultural preeminence
 According to our sources COUNTRY are focuse

COUNTRY think that COUNTRY compose a minor danger to COUNTRY strategies
COUNTRY think that COUNTRY have become a major problem to COUNTRY interests
COUNTRY think that the COUNTRY constitute a severe obstacle to COUNTRY activities
COUNTRY think that the COUNTRY have become a minor danger to COUNTRY ambitions
COUNTRYembassy in CITY STATE
COUNTRYembassy in COUNTRY
COUNTRYembassy in COUNTRY nation
COUNTRYembassy in The COUNTRY Nation
Charge daffaires in CITY STATE
Charge daffaires in COUNTRY
Charge daffaires in COUNTRY nation
Charge daffaires in The COUNTRY Nation
Cheerful news: we have ceased conflict with COUNTRY for now
Cheerful news: we have ended violence with COUNTRY for now
Cheerful news: we have ended war with COUNTRY for now
Cheerful news: we have halted war with COUNTRY for now
Cheerful news: we have stopped conflict with the COUNTRY for now
Cheerful news: we have terminated violence with COUNTRY for now
Cheerful news: we have terminated war with COUNTRY for now
Chief of Mission


Our reputed stance regarding the COUNTRY is likely friendly
Our reputed stance toward CITY STATE appears neutral
Our reputed stance with regard to CITY STATE is likely friendly
Our reputed stance with regard to COUNTRY looks friendly
Our reputed stance with regard to COUNTRY looks neutral
Our reputed stance with respect to the COUNTRY looks friendly
Our sources acknowledge COUNTRY as a favorable target
Our sources acknowledge COUNTRY as a soft target
Our sources acknowledge the COUNTRY as an average target
Our sources are acutely sure in this estimate
Our sources are much sure in this hypothesis
Our sources are not at all certain in this hypothesis
Our sources are not at all certain in this report
Our sources are not at all confident in this assessment
Our sources are not at all confident in this estimate
Our sources are not at all confident in this report
Our sources are not at all convinced in this assessment
Our sources are not at all convinced in this hypothesis
Our sources are not

The foreign office requests any material possible pertaining to COUNTRY economic potency
The foreign office requests any news available detailing COUNTRY economic potential
The foreign office requests any news available pertaining to COUNTRY economic might
The foreign office requests any news obtainable concerning COUNTRY military capability
The foreign office requests any news obtainable concerning COUNTRY military might
The foreign office requests any news on hand regarding COUNTRY military strength
The foreign office requests any news possible detailing COUNTRY economic potency
The foreign office requests any reporting available regarding COUNTRY economic power
The foreign office requests any reporting available relevant to COUNTRY economic capability
The foreign office requests any reporting in your possession regarding COUNTRY military potency
The foreign office requests any reporting obtainable detailing COUNTRY military power
The foreign office requests any reporting you have re

We imagine the COUNTRY to be a favorable party
We imagine the COUNTRY to be an archnemesis
We imagine the chance of conquering CITY STATE to be favorable
We imagine the chance of forcefully dominating CITY STATE to be average
We imagine the chance of forcefully dominating CITY STATE to be favorable
We imagine the chance of forcefully dominating COUNTRY to be average
We imagine the chance of forcefully dominating COUNTRY to be bad
We imagine the chance of taking over CITY STATE to be favorable
We imagine the chance of taking over CITY STATE to be soft
We imagine the chance of taking over the COUNTRY to be soft
We imagine the feasibility of conquering CITY STATE to be average
We imagine the feasibility of conquering CITY STATE to be favorable
We imagine the feasibility of conquering CITY STATE to be soft
We imagine the feasibility of conquering COUNTRY to be average
We imagine the feasibility of conquering COUNTRY to be bad
We imagine the feasibility of conquering the COUNTRY to be avera

In [38]:
important_phrases = {'is how':'friendship','is what we':'political disposition','Recent':'Military power'
                    }
possible_phrases = ['We are']

w1 = 'war'
w2 = ''
for i in dic.cable_types:
    if w1 in i:
        if w2 in i:
            print(i)
            print()

 An exchange was agreed upon: we shall provide COUNTRY an agreement to agree to a defensive pact for the subsequent 30 turns 2 units of iron for the subsequent 30 turns 2 units of oil for the subsequent 30 turns 2 units of aluminum for the subsequent 30 turns and 34 units of gold per turn for the subsequent 30 turns for a deal to join a mutual protection agreement for the upcoming 30 turns and a deal to declare war on COUNTRY

 Our public policy toward the COUNTRY appears deceptive

 We and COUNTRY signed on to a payment of an accord to stop the conflict for the next 10 turns and an accord to broker an armistice with CITY STATE for the next 10 turns for an accord to put an end to the war for the coming 10 turns an accord to broker an armistice with CITY STATE for the coming 10 turns an accord to broker an armistice with CITY STATE for the coming 10 turns an accord to broker an armistice with CITY STATE for the coming 10 turns and an accord to broker an armistice with CITY STATE for the